# Estructura de Prompts y Salidas para Agentes Robustos

La estructura adecuada de prompts y outputs es fundamental para crear agentes de IA más confiables y eficientes:

La estructura ayuda a:
- Mejora la consistencia en las respuestas
- Reduce ambigüedades en la comunicación
- Facilita el procesamiento de información

La implementación de estas estructuras resulta en agentes más robustos y predecibles en su comportamiento. La mayoría de plataformas se apoyan en una de las librerías clave para validar entradas en Python, [Pydantic](https://docs.pydantic.dev/latest/) que incluso tiene su apuesta personal en lo que a agentes se refiere: [PydanticAI](https://ai.pydantic.dev/).

Nosotros emplearemos para estos ejemplos el framework de Agno: https://docs.agno.com/agents/structured-output

In [6]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [7]:
import os
from openinference.instrumentation.agno import AgnoInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

# Set the endpoint and headers for LangSmith
endpoint = "https://api.smith.langchain.com/otel/v1/traces"
headers = {
    "x-api-key": os.getenv("LANGSMITH_API_KEY"),
    "Langsmith-Project": os.getenv("LANGSMITH_PROJECT"),
}

# Configure the tracer provider
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(
    SimpleSpanProcessor(OTLPSpanExporter(endpoint=endpoint, headers=headers))
)
trace_api.set_tracer_provider(tracer_provider=tracer_provider)

# Start instrumenting agno
AgnoInstrumentor().instrument()

In [8]:
from typing import List
from pydantic import BaseModel, Field

class MovieScript(BaseModel):
    setting: str = Field(..., description="Provide a nice setting for a blockbuster movie.")
    ending: str = Field(..., description="Ending of the movie. If not available, provide a happy ending.")
    genre: str = Field(
        ..., description="Genre of the movie. If not available, select action, thriller or romantic comedy."
    )
    name: str = Field(..., description="Give a name to this movie")
    characters: List[str] = Field(..., description="Name of characters for this movie.")
    storyline: str = Field(..., description="3 sentence storyline for the movie. Make it exciting!")

In [9]:
from agno.agent import Agent
from agno.models.google import Gemini

# Agent that uses structured outputs
structured_output_agent = Agent(
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    description="You write movie scripts.",
    response_model=MovieScript,
)

structured_output_agent.print_response("New York", stream=True)

Output()

El modo JSON es un modo por defecto que no implica la validación de los campos. Se emplea habitualmente como modo de fallback.

In [11]:
# Agent that uses JSON mode
json_mode_agent = Agent(
    model=Gemini(id="gemini-2.0-flash", temperature=0),
    description="You write movie scripts.",
    response_model=MovieScript,
    use_json_mode=True,
)
json_mode_agent.print_response("New York", stream=True)

Output()

Podríamos incluso hacer que un segundo modelo, no el principal, se encargue de la estructuración del texto.

```py
agent = Agent(
    model=Claude(id="claude-sonnet-4-20250514"),
    description="You write movie scripts.",
    response_model=MovieScript,
    parser_model=OpenAIChat(id="gpt-4o"),
)
```